### 필요 라이브러리 import

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords             #불용어
from urllib.request import urlretrieve
import zipfile
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [2]:
import numpy as np
import gensim
from urllib.request import urlretrieve, urlopen
import gzip
import zipfile
from gensim.models.word2vec import Word2Vec
import pandas as pd

In [3]:
import re

### 파일 불러오기

In [4]:
 #-*- coding: utf-8 -*- 
data=pd.read_csv("D:/test.csv", engine='python')
data.head()

,article_id,article_text,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,1,"우리는 많은 시간 인터넷을 이용하고 있지만, 이는 인터넷 회선을 통해 인터넷 서비스...",NaN,NaN,NaN
1,2,우리가 논의를 좀 더 전개하기 전에 먼저 이론적인 것 잠깐만 짚고 넘어가겠습니다. ...,NaN,NaN,NaN
2,3,GAN은 콘텐츠 생산 산업에 많은 변화를 가져올 것으로 보인다. 그러나 부정적인 측...,NaN,NaN,NaN
3,4,서양미술 혁신의 첫번째로 르네상스 미술 이야기부터 하겠습니다.\n한마디로 인간성의 ...,NaN,NaN,NaN
4,5,힘은 크기와 방향을 갖는 벡터양으로서 힘의 크기를 나타내는 SI 단위는 뉴턴(N)이...,NaN,NaN,NaN


In [5]:
data=data[['article_text']]
data['sentences']=data['article_text'].apply(sent_tokenize)
data['sentences'][0]

['우리는 많은 시간 인터넷을 이용하고 있지만, 이는 인터넷 회선을 통해 인터넷 서비스를 이용하는 것일 뿐, 내 컴퓨터 안의 파일이 인터넷에 연결되어 있는 것은 아닙니다.',
 'HTML로 웹사이트를 만들고 그 내용을 다른 사람들이 볼 수 있도록 하려면 HTML로 만든 웹 문서를 모두 서버 컴퓨터로 옮겨야 합니다.',
 '서버(server)컴퓨터란 전용선을 통해 인터넷에 직접 연결되어 있는 컴퓨터를 가리키는데, 24시간 인터넷에 연결되어있고 서버 컴퓨터 접속 주소만 알면 누구나 서버 컴퓨터의 내용을 볼 수 있습니다.',
 '인터넷 회선을 통해 서버 컴퓨터에 접속하는 사용자 컴퓨터를 클라이언트(client)컴퓨터라고 합니다.',
 '서버 정보를 가져와 보여주는 것은 사용자 컴퓨터 안의 웹 브라우저이기 때문에 좁은 의미로 웹브라우저를 클라이언트라고도 합니다.',
 '웹 디자이너나 웹 개발자들은 자신이 제작한 최신 웹사이트를 항상 서버 컴퓨터에 업로드해 놓기 때문에 사용자들은 자신의 위치에 상관없이 어디에서나 인터넷에 접속해서 해당 웹사이트의 내용을 볼 수 있습니다.',
 '개인은 웹 서버를 마련하기 어렵기 때문에 서버의 일부 공간을 매달 혹은 몇 년마다 일정 금액을 내고 사용하는 서비스를 이용합니다.',
 "이것을 '서버 호스팅 서비스' 혹은 '웹 호스팅 서비스'라고 하는데, 개인 웹사이트를 운영하는 사람들은 대부분 이런 호스팅 서비스를 이용합니다.",
 '호스팅 서비스는 어떤 서버를 이용하느냐에 따라 윈도우 서버 호스팅과 리눅스 서버 호스팅으로 나뉘는데, 윈도우 서버에서는 ASP나 ASAP.NET 프로그래밍 언어를 사용하고, 리눅스 서버에서는 PHP 프로그래밍 언어를 사용하며 좀 더 대중적이고 저렴합니다.']

In [6]:
data

,article_text,sentences
0,"우리는 많은 시간 인터넷을 이용하고 있지만, 이는 인터넷 회선을 통해 인터넷 서비스...","[우리는 많은 시간 인터넷을 이용하고 있지만, 이는 인터넷 회선을 통해 인터넷 서비..."
1,우리가 논의를 좀 더 전개하기 전에 먼저 이론적인 것 잠깐만 짚고 넘어가겠습니다. ...,[우리가 논의를 좀 더 전개하기 전에 먼저 이론적인 것 잠깐만 짚고 넘어가겠습니다....
2,GAN은 콘텐츠 생산 산업에 많은 변화를 가져올 것으로 보인다. 그러나 부정적인 측...,"[GAN은 콘텐츠 생산 산업에 많은 변화를 가져올 것으로 보인다., 그러나 부정적인..."
3,서양미술 혁신의 첫번째로 르네상스 미술 이야기부터 하겠습니다.\n한마디로 인간성의 ...,"[서양미술 혁신의 첫번째로 르네상스 미술 이야기부터 하겠습니다., 한마디로 인간성의..."
4,힘은 크기와 방향을 갖는 벡터양으로서 힘의 크기를 나타내는 SI 단위는 뉴턴(N)이...,[힘은 크기와 방향을 갖는 벡터양으로서 힘의 크기를 나타내는 SI 단위는 뉴턴(N)...


### word2vec 파일 가져오기 (더 좋은 파일 있으면 대체하기)

In [7]:
kovec=Word2Vec.load("D:\ko.bin")
#kovec=gensim.models.KeyedVectors.load_word2vec_format('D:\ko.bin',binary=True)

In [8]:
model=Word2Vec(data,sg=1,size=100,window=3,min_count=3,workers=4)

### 불용어 추가 및 전처리

In [9]:
stop_words=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def preprocess_sentence(sentence):
    sentence=[re.sub(r'[^가-힣\s]','',word) for word in sentence]
    return [word for word in sentence if word not in stop_words and word]

def preprocess_sentences(sentences):
    return [preprocess_sentence(sentence) for sentence in sentences]
#a-zA-Z, 즉 영어는 포함 안되게 했음

In [10]:
## 형태소 단위로 Tokenize
from konlpy.tag import Okt  
okt=Okt()

def tokenization(sentences):
    temp=[]
    for sentence in sentences:
        temp_X=okt.morphs(sentence,stem=True)
        temp_X=[word for word in temp_X]
        temp.append(temp_X)
    return temp


data['tokenized_data']=data['sentences'].apply(tokenization)
    
data['tokenized_data']=data['tokenized_data'].apply(preprocess_sentences)
    
print(data)

                                        article_text  \
0  우리는 많은 시간 인터넷을 이용하고 있지만, 이는 인터넷 회선을 통해 인터넷 서비스...   
1  우리가 논의를 좀 더 전개하기 전에 먼저 이론적인 것 잠깐만 짚고 넘어가겠습니다. ...   
2  GAN은 콘텐츠 생산 산업에 많은 변화를 가져올 것으로 보인다. 그러나 부정적인 측...   
3  서양미술 혁신의 첫번째로 르네상스 미술 이야기부터 하겠습니다.\n한마디로 인간성의 ...   
4  힘은 크기와 방향을 갖는 벡터양으로서 힘의 크기를 나타내는 SI 단위는 뉴턴(N)이...   

                                           sentences  \
0  [우리는 많은 시간 인터넷을 이용하고 있지만, 이는 인터넷 회선을 통해 인터넷 서비...   
1  [우리가 논의를 좀 더 전개하기 전에 먼저 이론적인 것 잠깐만 짚고 넘어가겠습니다....   
2  [GAN은 콘텐츠 생산 산업에 많은 변화를 가져올 것으로 보인다., 그러나 부정적인...   
3  [서양미술 혁신의 첫번째로 르네상스 미술 이야기부터 하겠습니다., 한마디로 인간성의...   
4  [힘은 크기와 방향을 갖는 벡터양으로서 힘의 크기를 나타내는 SI 단위는 뉴턴(N)...   

                                      tokenized_data  
0  [[우리, 많다, 시간, 인터넷, 을, 이용, 하고, 있다, 이다, 인터넷, 회선,...  
1  [[우리, 논의, 더, 전개, 전, 먼저, 이론, 적, 인, 것, 잠깐, 만, 짚다...  
2  [[콘텐츠, 생산, 산업, 많다, 변화, 가져오다, 것, 보이다], [그러나, 부정...  
3  [[서양미술, 혁신, 첫, 번째, 로, 르네상스, 미술, 이야기, 부터], [한마디...  
4  [[힘, 크기, 방향, 을, 갖다, 벡터, 양, 으로서, 힘, 크기, 나타내다, 단..

In [11]:
embedding_dim=200
zero_vector=np.zeros(embedding_dim)

### 문장별 벡터값 구하기

In [12]:
def calculate_sentence_vector(sentence):
    if len(sentence) != 0:
        sum=0
        for word in sentence:
            try: 
                sum+=kovec.wv[word]
            except Exception:
                sum+=zero_vector
        return sum/len(sentence)
    else:
        return zero_vector

In [13]:
def sentences_to_vectors(sentences):
    return [calculate_sentence_vector(sentence) for sentence in sentences]

In [14]:
data['SentenceEmbedding']=data['tokenized_data'].apply(sentences_to_vectors)
data[['SentenceEmbedding']]

,SentenceEmbedding
0,"[[-0.19131963, 0.002412068, -0.478244, 0.49261..."
1,"[[-0.05495586, -0.2989531, 0.074204944, 0.0287..."
2,"[[0.23017685, -0.068566605, 0.19649297, 0.6754..."
3,"[[-0.7269085413879819, -0.9547493942081928, 0...."
4,"[[0.14874788, 0.07012593, -0.24334846, -0.4889..."


### 문장별 유사도 구하기

In [15]:
def similarity_matrix(sentence_embedding):
    sim_mat=np.zeros([len(sentence_embedding),len(sentence_embedding)])
    for i in range(len(sentence_embedding)):
        for j in range(len(sentence_embedding)):
            sim_mat[i][j]=cosine_similarity(sentence_embedding[i].reshape(1,embedding_dim),
                                           sentence_embedding[j].reshape(1,embedding_dim))[0,0]
    return sim_mat

In [16]:
data['SimMatrix']=data['SentenceEmbedding'].apply(similarity_matrix)
data['SimMatrix']

0    [[1.0000001192092896, 0.6782692074775696, 0.88...
1    [[1.0, 0.5473900547815026, 0.28144019842147827...
2    [[1.0000001192092896, 0.2837713360786438, 0.33...
3    [[1.0, 0.3556040231558631, 0.20608101207169816...
4    [[1.0000001192092896, 0.813560962677002, 0.714...
Name: SimMatrix, dtype: object

def draw_graphs(sim_matrix):
    nx_graph=nx.from_numpy_array(sim_matrix)
    plt.figure(figsize=(10,10))
    pos=nx.spring_layout(nx_graph)
    nx.draw(nx_graph,with_labels=True,font_weight='bold')
    nx.draw_networkx_edge_labels(nx_graph,pos,font_color='red')
    plt.show()

draw_graphs(data['SimMatrix'][0])

### 점수 계산

In [18]:
def calculate_score(sim_matrix):
    nx_graph=nx.from_numpy_array(sim_matrix)
    scores=nx.pagerank_numpy(nx_graph)
    return scores

In [19]:
data['score']=data['SimMatrix'].apply(calculate_score)
data[['SimMatrix','score']]

,SimMatrix,score
0,"[[1.0000001192092896, 0.6782692074775696, 0.88...","{0: 0.11228407622699133, 1: 0.1038480801169705..."
1,"[[1.0, 0.5473900547815026, 0.28144019842147827...","{0: 0.05687100044764587, 1: 0.0625569719751471..."
2,"[[1.0000001192092896, 0.2837713360786438, 0.33...","{0: 0.04406322355924085, 1: 0.0316508886299722..."
3,"[[1.0, 0.3556040231558631, 0.20608101207169816...","{0: 0.08598122421458233, 1: 0.0932427771800998..."
4,"[[1.0000001192092896, 0.813560962677002, 0.714...","{0: 0.06560539019924444, 1: 0.0679637572320210..."


In [20]:
data['score'][0]

{0: 0.11228407622699133,
 1: 0.10384808011697051,
 2: 0.11604758798046202,
 3: 0.11522323151704224,
 4: 0.10945307875867902,
 5: 0.11493492253322914,
 6: 0.10619924547368009,
 7: 0.11320730828755055,
 8: 0.1088024691053952}

In [38]:
def ranked_sentences(sentences,scores,n=7):
    top_scores=sorted(((scores[i],s)
                      for i,s in enumerate(sentences)),
                     reverse=True)
    top_n_sentences=[sentence for score,sentence in top_scores[:n]]
    return " ".join(top_n_sentences)

### 요약 결과값 산출

In [40]:
data['summary'] = data.apply(lambda x:
                            ranked_sentences(x.sentences, x.score),axis=1)

In [41]:
for i in range(0,len(data)):
    print(i+1,'번 문서')
    print('원문 : ',data.loc[i].article_text)
    print(' ')
    print('요약 : ',data.loc[i].summary)
    print('')

1 번 문서
원문 :  우리는 많은 시간 인터넷을 이용하고 있지만, 이는 인터넷 회선을 통해 인터넷 서비스를 이용하는 것일 뿐, 내 컴퓨터 안의 파일이 인터넷에 연결되어 있는 것은 아닙니다. HTML로 웹사이트를 만들고 그 내용을 다른 사람들이 볼 수 있도록 하려면 HTML로 만든 웹 문서를 모두 서버 컴퓨터로 옮겨야 합니다. 서버(server)컴퓨터란 전용선을 통해 인터넷에 직접 연결되어 있는 컴퓨터를 가리키는데, 24시간 인터넷에 연결되어있고 서버 컴퓨터 접속 주소만 알면 누구나 서버 컴퓨터의 내용을 볼 수 있습니다. 
인터넷 회선을 통해 서버 컴퓨터에 접속하는 사용자 컴퓨터를 클라이언트(client)컴퓨터라고 합니다. 서버 정보를 가져와 보여주는 것은 사용자 컴퓨터 안의 웹 브라우저이기 때문에 좁은 의미로 웹브라우저를 클라이언트라고도 합니다. 
웹 디자이너나 웹 개발자들은 자신이 제작한 최신 웹사이트를 항상 서버 컴퓨터에 업로드해 놓기 때문에 사용자들은 자신의 위치에 상관없이 어디에서나 인터넷에 접속해서 해당 웹사이트의 내용을 볼 수 있습니다. 
개인은 웹 서버를 마련하기 어렵기 때문에 서버의 일부 공간을 매달 혹은 몇 년마다 일정 금액을 내고 사용하는 서비스를 이용합니다. 이것을 '서버 호스팅 서비스' 혹은 '웹 호스팅 서비스'라고 하는데, 개인 웹사이트를 운영하는 사람들은 대부분 이런 호스팅 서비스를 이용합니다.
호스팅 서비스는 어떤 서버를 이용하느냐에 따라 윈도우 서버 호스팅과 리눅스 서버 호스팅으로 나뉘는데, 윈도우 서버에서는 ASP나 ASAP.NET 프로그래밍 언어를 사용하고, 리눅스 서버에서는 PHP 프로그래밍 언어를 사용하며 좀 더 대중적이고 저렴합니다. 
 
요약 :  서버(server)컴퓨터란 전용선을 통해 인터넷에 직접 연결되어 있는 컴퓨터를 가리키는데, 24시간 인터넷에 연결되어있고 서버 컴퓨터 접속 주소만 알면 누구나 서버 컴퓨터의 내용을 볼 수 있습니다. 인터넷 회선을 통해 서버 컴퓨터에 접속하는 사용자 컴퓨터를 클라이언트(clie

# 문제 만들기

### 문장 생성에 쓸 중요 단어 구하기 위한 작업

In [42]:
data['summary_sentences']=data['summary'].apply(sent_tokenize)
data['summary_sentences'][0]

['서버(server)컴퓨터란 전용선을 통해 인터넷에 직접 연결되어 있는 컴퓨터를 가리키는데, 24시간 인터넷에 연결되어있고 서버 컴퓨터 접속 주소만 알면 누구나 서버 컴퓨터의 내용을 볼 수 있습니다.',
 '인터넷 회선을 통해 서버 컴퓨터에 접속하는 사용자 컴퓨터를 클라이언트(client)컴퓨터라고 합니다.',
 '웹 디자이너나 웹 개발자들은 자신이 제작한 최신 웹사이트를 항상 서버 컴퓨터에 업로드해 놓기 때문에 사용자들은 자신의 위치에 상관없이 어디에서나 인터넷에 접속해서 해당 웹사이트의 내용을 볼 수 있습니다.',
 "이것을 '서버 호스팅 서비스' 혹은 '웹 호스팅 서비스'라고 하는데, 개인 웹사이트를 운영하는 사람들은 대부분 이런 호스팅 서비스를 이용합니다.",
 '우리는 많은 시간 인터넷을 이용하고 있지만, 이는 인터넷 회선을 통해 인터넷 서비스를 이용하는 것일 뿐, 내 컴퓨터 안의 파일이 인터넷에 연결되어 있는 것은 아닙니다.',
 '서버 정보를 가져와 보여주는 것은 사용자 컴퓨터 안의 웹 브라우저이기 때문에 좁은 의미로 웹브라우저를 클라이언트라고도 합니다.',
 '호스팅 서비스는 어떤 서버를 이용하느냐에 따라 윈도우 서버 호스팅과 리눅스 서버 호스팅으로 나뉘는데, 윈도우 서버에서는 ASP나 ASAP.NET 프로그래밍 언어를 사용하고, 리눅스 서버에서는 PHP 프로그래밍 언어를 사용하며 좀 더 대중적이고 저렴합니다.']

In [43]:
data['tokenized_data_for_word']=data['summary_sentences'].apply(tokenization)
    
data['tokenized_data_for_word']=data['tokenized_data_for_word'].apply(preprocess_sentences)

def list_to_string(tokenized_data):
    tmp=[]
    for token_sentence in tokenized_data:
        for token in token_sentence:
            tmp.append(token)
    return tmp

data['token']=data['tokenized_data_for_word'].apply(list_to_string)

data['token']

0    [서버, 컴퓨터, 란, 전용선, 을, 통해, 인터넷, 직접, 연결, 되어다, 있다,...
1    [귀납, 과거, 사례, 라든지, 축적, 되다, 데이터, 분석, 그, 안, 에서, 보...
2    [해당, 영상, 에는, 미국, 전, 대통령, 버락, 오바마, 집무실, 배경, 도널드...
3    [사실, 상, 서양, 미술사, 에서, 가장, 획기, 적, 인, 사건, 평면, 위, ...
4    [중력, 질량, 있다, 물체, 사이, 작용, 힘, 으로서, 만유인력, 이라고도, 앞...
Name: token, dtype: object

### summary 내의 단어들에 대한 점수 구하기

In [57]:
def calculate_word_vector(word):                        
    if len(word) != 0:                        
        sum=0                        
        for token in word:
            try: 
                sum+=kovec.wv[token]
            except Exception:
                sum+=zero_vector
        return sum
    else:
        return zero_vector

def words_to_vectors(words):
    return [calculate_word_vector(word) for word in words]

data['word_value']=data['token'].apply(words_to_vectors)
data['word_value']

data['SimMatrix_word']=data['word_value'].apply(similarity_matrix)
data['SimMatrix_word']

data['word_score']=data['SimMatrix_word'].apply(calculate_score)
data['word_score']



0    {0: -2072337172959.853, 1: -1335681052378.5742...
1    {0: 1189318012022.2666, 1: 8838910641710.56, 2...
2    {0: 621801330570984.2, 1: 1099655828618137.8, ...
3    {0: -40475531802869.016, 1: -1449599601450.140...
4    {0: -38247218842.43525, 1: -312733097593.92474...
Name: word_score, dtype: object

In [60]:
def ranked_words(tokenized_data, scores, n=100):
    top_scores=sorted(((scores[i],s)
                      for i,s in enumerate(tokenized_data)),
                     reverse=True)
    #top_n_sentences=[sentence for score,sentence in top_scores[:n]]
    top_n_sentences=[]
    for score, word in top_scores:
        if word not in top_n_sentences:
            top_n_sentences.append(word)
    return " ".join(top_n_sentences)

data['important_word'] = data.apply(lambda x:
                           ranked_words(x.token, x.word_score),axis=1)
import sys
np.set_printoptions(threshold=sys.maxsize)
word_set = data['important_word'][2]

### 형태소 분석을 이용한 빈칸뚫기

In [61]:
import urllib3
import json
import requests
import sys

openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU" 

openApiURL2 = "http://aiopen.etri.re.kr:8000/WiseNLU_spoken"
 
accessKey = "d4a16891-dd45-4883-a873-777a8fda8787"
analysisCode = "ner"
text = ""
 
result=""
text += word_set

requestJson = {
    "access_key": accessKey,
    "argument": {
        "text": text,
        "analysis_code": analysisCode
    }
}
 
http = urllib3.PoolManager()
response = http.request(
    "POST",
    openApiURL,
    headers={"Content-Type": "application/json; charset=UTF-8"},
    body=json.dumps(requestJson)
)

 
print("[responseCode] " + str(response.status))
print("[responBody]")


data_2 = json.loads(str(response.data,"utf-8")) # create json object using string

sentence=data_2['return_object']['sentence'] # get the data of sentences

for s in sentence: 
    for w in s['word']: #  for loop : words

        text=w['text'] # the text of word

        text_id=int(w['id']) # the id of word

        begin=int(w['begin']) # the beginning index of morphemes in word

        end=int(w['end']) # the ending index of morphemes in word

    
        for i in range(begin, end+1): # for : morphemes

            lemma=s['morp'][i]['lemma'] # morpheme

            pos=s['morp'][i]['type'] # tag
            ''''sys.stdout.write("[")'''
            if pos=="NNG":
                #sys.stdout.write("일반명사")
                sys.stdout.write(lemma+"\n")
                result +=lemma+" "
            if pos=="NNP":
                #sys.stdout.write("고유명사")
                sys.stdout.write(lemma+"\n")
                result +=lemma+" "
'''
            if pos=="NNB":
                sys.stdout.write("의존명사")
            if pos=="NP":
                sys.stdout.write("대명사")
            if pos=="NR":
                sys.stdout.write("수사")
            if pos=="VV":
                sys.stdout.write("동사")
            if pos=="VA":
                sys.stdout.write("형용사")
            if pos=="VX":
                sys.stdout.write("보조용언")
            if pos=="VCP":
                sys.stdout.write("긍정지정사")
            if pos=="VCN":
                sys.stdout.write("부정지정사")
            if pos=="MM":
                sys.stdout.write("관형사")
            if pos=="MAG":
                sys.stdout.write("일반부사")
            if pos=="MAJ":
                sys.stdout.write("접속부사")
            if pos=="IC":
                sys.stdout.write("감탄사")
            if pos=="JKS":
                sys.stdout.write("주격조사")
            if pos=="JKC":
                sys.stdout.write("보격조사")
            if pos=="JKG":
                sys.stdout.write("관형격조사")
            if pos=="JKO":
                sys.stdout.write("목적격조사")
            if pos=="JKB":
                sys.stdout.write("부사격조사")
            if pos=="JKV":
                sys.stdout.write("호격조사")
            if pos=="JKQ":
                sys.stdout.write("인용격조사")
            if pos=="JX":
                sys.stdout.write("보격조사")
            if pos=="JC":
                sys.stdout.write("접속조사")
            if pos=="EP":
                sys.stdout.write("선어말어미")
            if pos=="EF":
                sys.stdout.write("종결어미")
            if pos=="EC":
                sys.stdout.write("연결어미")
            if pos=="ETN":
                sys.stdout.write("명사형전성어미")
            if pos=="ETM":
                sys.stdout.write("관형형전성어미")
            if pos=="XPN":
                sys.stdout.write("체언접두사")
            if pos=="XSN":
                sys.stdout.write("명사파생접미사")
            if pos=="XSV":
                sys.stdout.write("동사파생접미사")
            if pos=="XSA":
                sys.stdout.write("형용사파생접미사")
            if pos=="XR":
                sys.stdout.write("어근")
            if pos=="SF":
                sys.stdout.write("마침표,물음표,느낌표")
            if pos=="SP":
                sys.stdout.write("쉼표,가운데점,콜론,빗금")
            if pos=="SS":
                sys.stdout.write("따옴표,괄호표,줄표")
            if pos=="SE":
                sys.stdout.write("줄임표")
            if pos=="SO":
                sys.stdout.write("붙임표")
            if pos=="SL":
                sys.stdout.write("외국어")
            if pos=="SH":
                sys.stdout.write("한자")
            if pos=="SW":
                sys.stdout.write("기타기호")
            if pos=="NF":
                sys.stdout.write("명사추정범주")
            if pos=="NV":
                sys.stdout.write("용언추정범주")
            if pos=="SN":
                sys.stdout.write("숫자")
            if pos=="NA":
                sys.stdout.write("분석불능범주") 
            sys.stdout.write("]")    
            sys.stdout.write(lemma+"/") # print them
            
            

            if i<end: # print + if the morpheme is not the last one

                sys.stdout.write(" + ")

        print()

    print()
    

for s in sentence: 
    t=0
    for w in s['NE']: #  for loop : NE

        text=w['text']; # the text of word

        text_id=int(w['id']); # the id of word

        begin=int(w['begin']) # the beginning index of morphemes in word

        end=int(w['end']) # the ending index of morphemes in word
        
    
        for i in range(begin, end+1): # for : morphemes
            
            ga=s['NE'][t]['type']
            t+=1
            sys.stdout.write(text+" : ")
            
            if ga=="PS_NAME":
                sys.stdout.write("사람 이름")
            if ga=="LC_OTHERS":
                sys.stdout.write("기타 장소")
            if ga=="LCP_COUNTRY":
                sys.stdout.write("국가명")
            if ga=="LCP_PROVINCE":
                sys.stdout.write("도, 주 지역명")
            if ga=="LCP_COUNTY":
                sys.stdout.write("세부 행정구역명")
            if ga=="LCP_CITY":
                sys.stdout.write("도시명")
            if ga=="LCP_CAPITALCITY":
                sys.stdout.write("수도명")
            if ga=="LCG_RIVER":
                sys.stdout.write("강")
            if ga=="LCG_OCEAN":
                sys.stdout.write("바다")
            if ga=="LCG_MOUNTAIN":
                sys.stdout.write("산")
            if ga=="LCG_CONTINENT":
                sys.stdout.write("대륙")
            if ga=="LC_SPACE":
                sys.stdout.write("천체 명칭")
            if ga=="LCG_ISLAND":
                sys.stdout.write("섬")

            if ga=="OG_OTHERS":
                sys.stdout.write("기타 기관/단체")
            if ga=="OGG_ECONOMY":
                sys.stdout.write("경제 관련 기관")
            if ga=="OGG_EDUCATION":
                sys.stdout.write("교육 기관")
            if ga=="OGG_MILITARY":
                sys.stdout.write("군사 기관/")
            if ga=="OGG_MEDIA":
                sys.stdout.write("미디어 기관")
            if ga=="OGG_SPORTS":
                sys.stdout.write("스포츠 기관")
            if ga=="OGG_ART	":
                sys.stdout.write("예술 기관")
            if ga=="OGG_MEDICINE":
                sys.stdout.write("의료 기관")
            if ga=="OGG_RELIGION":
                sys.stdout.write("종교 기관")
            if ga=="OGG_SCIENCE":
                sys.stdout.write("과학 기관")
            if ga=="OGG_LIBRARY":
                sys.stdout.write("도서관")
            if ga=="OGG_LAW":
                sys.stdout.write("법률 기관")
            if ga=="OGG_POLITICS":
                sys.stdout.write("정부/행정 기관")
            if ga=="OGG_FOOD":
                sys.stdout.write("음식 업체")
            if ga=="OGG_HOTEL":
                sys.stdout.write("숙박 업체")
            if ga=="DT_OTHERS":
                sys.stdout.write("기타 날짜")
            if ga=="DT_DURATION":
                sys.stdout.write("기간")
            if ga=="DT_DAY":
                sys.stdout.write("날짜/절기")
            if ga=="DT_MONTH":
                sys.stdout.write("달")
            if ga=="DT_YEAR":
                sys.stdout.write("년")
            if ga=="DT_SEASON":
                sys.stdout.write("계절")
            if ga=="DT_GEOAGE":
                sys.stdout.write("지질시대")
            if ga=="DT_DYNASTY":
                sys.stdout.write("왕조시대")
            if ga=="TI_OTHERS":
                sys.stdout.write("기타 시간")
            if ga=="TI_DURATION":
                sys.stdout.write("기간")
            if ga=="TI_HOUR":
                sys.stdout.write("시각")
            if ga=="TI_MINUTE":
                sys.stdout.write("분")
            if ga=="TI_SECOND":
                sys.stdout.write("초")
            if ga=="CV_NAME":
                sys.stdout.write("문명/문화")
            if ga=="CV_TRIBE":
                sys.stdout.write("민족/종족")
            if ga=="CV_SPORTS":
                sys.stdout.write("스포츠")
            if ga=="CV_SPORTS_INST":
                sys.stdout.write("스포츠 용품")
            if ga=="CV_POLICY":
                sys.stdout.write("제도/정책")
            if ga=="CV_TAX":
                sys.stdout.write("조세")
            if ga=="CV_FUNDS":
                sys.stdout.write("기금")
            if ga=="CV_LANGUAGE":
                sys.stdout.write("언어")
            if ga=="CV_BUILDING_TYPE":
                sys.stdout.write("건축양식")
            if ga=="CV_FOOD	":
                sys.stdout.write("음식")
            if ga=="CV_DRINK":
                sys.stdout.write("음료수")
            if ga=="CV_CLOTHING":
                sys.stdout.write("의복")
            if ga=="CV_POSITION":
                sys.stdout.write("직위")
            if ga=="CV_RELATION":
                sys.stdout.write("인간 관계")
            if ga=="CV_OCCUPATION":
                sys.stdout.write("직업")
            if ga=="CV_CURRENCY":
                sys.stdout.write("통화")
            if ga=="CV_PRIZE":
                sys.stdout.write("상")
            if ga=="CV_LAW":
                sys.stdout.write("법/법률")
            if ga=="CV_FOOD_STYLE":
                sys.stdout.write("음식 종류")
            if ga=="AM_OTHERS":
                sys.stdout.write("기타 동물")
            if ga=="AM_INSECT":
                sys.stdout.write("곤충")
            if ga=="AM_BIRD":
                sys.stdout.write("조류")
            if ga=="AM_FISH":
                sys.stdout.write("어류")
            if ga=="AM_MAMMALIA":
                sys.stdout.write("포유류")
            if ga=="AM_AMPHIBIA":
                sys.stdout.write("양서류")
            if ga=="AM_REPTILIA":
                sys.stdout.write("파충류")
            if ga=="AM_TYPE":
                sys.stdout.write("동물 분류")
            if ga=="AM_PART":
                sys.stdout.write("신체 부위")
            if ga=="PT_OTHERS":
                sys.stdout.write("기타 식물")
            if ga=="PT_FRUIT":
                sys.stdout.write("과일")
            if ga=="PT_FLOWER":
                sys.stdout.write("꽃")
            if ga=="PT_TREE":
                sys.stdout.write("나무")
            if ga=="PT_GRASS":
                sys.stdout.write("풀")
            if ga=="PT_TYPE":
                sys.stdout.write("식물 유형")
            if ga=="PT_PART":
                sys.stdout.write("식물의 한 부분")

            if ga=="QT_OTHERS":
                sys.stdout.write("기타 수량")
            if ga=="QT_AGE":
                sys.stdout.write("나이")
            if ga=="QT_SIZE":
                sys.stdout.write("크기/넓이")
            if ga=="QT_LENGTH":
                sys.stdout.write("길이/거리/높이")
            if ga=="QT_COUNT":
                sys.stdout.write("개수/빈도")
            if ga=="QT_MAN_COUNT":
                sys.stdout.write("인원수")
            if ga=="QT_WEIGHT":
                sys.stdout.write("무게")
            if ga=="QT_PERCENTAGE":
                sys.stdout.write("비율")
            if ga=="QT_SPEED":
                sys.stdout.write("속도")
            if ga=="QT_TEMPERATURE":
                sys.stdout.write("")
            if ga=="QT_VOLUME":
                sys.stdout.write("부피")
            if ga=="QT_ORDER":
                sys.stdout.write("순서")
            if ga=="QT_PRICE":
                sys.stdout.write("금액")
            if ga=="QT_PHONE":
                sys.stdout.write("전화번호")
            if ga=="QT_SPORTS":
                sys.stdout.write("스포츠 관련 수량")
            if ga=="QT_CHANNEL":
                sys.stdout.write("채널")
            if ga=="QT_ALBUM":
                sys.stdout.write("앨범 관련 수량")
            if ga=="QT_ZIPCODE":
                sys.stdout.write("우편번호")
            if ga=="FD_OTHERS":
                sys.stdout.write("과학 학문")
            if ga=="FD_SCIENCE":
                sys.stdout.write("사회과학 학문")
            if ga=="FD_SOCIAL_SCIENCE":
                sys.stdout.write("정치/경제/사회 학문")
            if ga=="FD_MEDICINE":
                sys.stdout.write("의학 관련 학문")
            if ga=="FD_ART":
                sys.stdout.write("예술관련 학문")
            if ga=="FD_PHILOSOPHY":
                sys.stdout.write("철학 관련 학문")

            if ga=="TR_OTHERS":
                sys.stdout.write("기타이론")
            if ga=="TR_SCIENCE":
                sys.stdout.write("과학 관련 이론")
            if ga=="TR_SOCIAL_SCIENCE":
                sys.stdout.write("사회과학 이론")
            if ga=="TR_ART":
                sys.stdout.write("예술관련 이론")
            if ga=="TR_PHILOSOPHY":
                sys.stdout.write("철학 이론")
            if ga=="TR_MEDICINE":
                sys.stdout.write("의학 진단법")

            if ga=="EV_OTHERS":
                sys.stdout.write("기타 사건")
            if ga=="EV_ACTIVITY":
                sys.stdout.write("사회운동")
            if ga=="EV_WAR_REVOLUTION":
                sys.stdout.write("전쟁/혁명")
            if ga=="EV_SPORTS":
                sys.stdout.write("스포츠/레저")
            if ga=="EV_FESTIVAL":
                sys.stdout.write("축제")
            if ga=="MT_ELEMENT":
                sys.stdout.write("원소명")
            if ga=="MT_METAL":
                sys.stdout.write("금속물")
            if ga=="MT_ROCK":
                sys.stdout.write("암석")
            if ga=="MT_CHEMICAL":
                sys.stdout.write("화학물질")
            if ga=="TM_COLOR":
                sys.stdout.write("색")
            if ga=="TM_DIRECTION":
                sys.stdout.write("방향")
            if ga=="TM_CLIMATE":
                sys.stdout.write("기후지역")
            if ga=="TM_SHAPE":
                sys.stdout.write("모양/형태")
            if ga=="TM_CELL_TISSUE":
                sys.stdout.write("세포/조직/기관")
            if ga=="TMM_DISEASE":
                sys.stdout.write("증세/질병")
            if ga=="TMM_DRUG":
                sys.stdout.write("약/약품명")
            if ga=="TMI_HW":
                sys.stdout.write("IT 하드웨어")
            if ga=="TMI_SW":
                sys.stdout.write("IT 소프트웨어")
            if ga=="TMI_SITE":
                sys.stdout.write("URL 주소")
            if ga=="TMI_EMAIL":
                sys.stdout.write("이메일주소")
            if ga=="TMI_MODEL":
                sys.stdout.write("모델명, 부품류")
            if ga=="TMI_SERVICE":
                sys.stdout.write("IT 서비스")
            if ga=="TMI_PROJECT":
                sys.stdout.write("프로젝트")
            if ga=="TMIG_GENRE":
                sys.stdout.write("게임 장르")
            if ga=="TM_SPORTS":
                sys.stdout.write("스포츠/레저")
            if ga=="AFW_DOCUMENT":
                sys.stdout.write("서적")
            if ga=="AFW_PERFORMANCE":
                sys.stdout.write("가극")
            if ga=="AFW_VIDEO":
                sys.stdout.write("비디오")
            if ga=="AFW_ART_CRAFT":
                sys.stdout.write("미술작품")
            if ga=="AFW_MUSIC":
                sys.stdout.write("음악작품")
            if ga=="AF_WARES":
                sys.stdout.write("상품")
            if ga=="AF_TRANSPORT":
                sys.stdout.write("운송수단")
            if ga=="AF_WEAPON":
                sys.stdout.write("무기")
            if ga=="AF_MUSICAL_INSTRUMENT":
                sys.stdout.write("악기")
            if ga=="AF_ROAD":
                sys.stdout.write("도로")
            if ga=="AF_BUILDING":
                sys.stdout.write("건축물")
            if ga=="AF_CULTURAL_ASSET":
                sys.stdout.write("문화재")
            if ga=="AF_WORKS":
                sys.stdout.write("기타작품")
            if i<end:
                break
        print()
'''
print()

[responseCode] 200
[responBody]
월
대응
대통령
일
요인
유튜브
포르노
경쟁
회로
오바마
콘텐츠
제작
실제
마이크로소프트
트럼프
트레이스
처벌
영상
비판
페이스북
미국
버락
배경
버즈피드
생성자
연구
집무
제작
배포
확장
해당
알고리즘
딥
정권
경고
진짜
진행
페이크
시행
국내
구글
위험
발견
위
법
식별
도널드
개
식별
필요
강화



In [62]:
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords             #불용어
from urllib.request import urlretrieve
import zipfile
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import numpy as np
import gensim
from urllib.request import urlretrieve, urlopen
import gzip
import zipfile
from gensim.models.word2vec import Word2Vec
import pandas as pd
from gensim.test.utils import common_texts

### 위에서 추린 단어들을 사용자가 입력한 텍스트 전반의 키워드 중심으로 중요도 재배열

In [63]:
kovec=Word2Vec.load("D:\ko.bin")
print(type(kovec))
#model=Word2Vec(data,sg=1,size=100,window=3,min_count=3,workers=4)
#kovec.wv.most_similar("르네상스")
arr = result.split(" ")
dict={}

for word in arr:
    try:
        dict[word]=kovec.wv.similarity("범죄",word)
    except:
        dict[word]=0
        
import operator
sdict= sorted(dict.items(), key=operator.itemgetter(1), reverse=True)
print(sdict)

<class 'gensim.models.word2vec.Word2Vec'>
[('처벌', 0.57618254), ('위험', 0.37469196), ('포르노', 0.32359743), ('법', 0.3183799), ('요인', 0.30952168), ('경고', 0.2800363), ('비판', 0.24263857), ('콘텐츠', 0.2322104), ('대응', 0.22861002), ('실제', 0.22842626), ('연구', 0.22263357), ('국내', 0.21246625), ('해당', 0.20864704), ('필요', 0.2000242), ('경쟁', 0.19720986), ('식별', 0.16379878), ('강화', 0.14771946), ('배포', 0.14526124), ('정권', 0.14089127), ('미국', 0.1183561), ('배경', 0.11684337), ('알고리즘', 0.11462056), ('시행', 0.10222533), ('회로', 0.10102601), ('대통령', 0.10026897), ('진짜', 0.098653734), ('영상', 0.09785205), ('개', 0.09257738), ('위', 0.047064636), ('일', 0.0403504), ('집무', 0.039007805), ('진행', 0.027587363), ('확장', 0.027098611), ('버락', 0.013818292), ('유튜브', 0), ('오바마', 0), ('마이크로소프트', 0), ('트레이스', 0), ('페이스북', 0), ('버즈피드', 0), ('생성자', 0), ('페이크', 0), ('구글', 0), ('도널드', 0), ('', 0), ('제작', -0.0003935084), ('딥', -0.014643257), ('트럼프', -0.021699859), ('발견', -0.03129503), ('월', -0.15243451)]


In [68]:
blank_arr = []
for idx in range(len(sdict)):
    blank_arr.append(sdict[idx][0])

In [69]:
print(blank_arr)

['처벌', '위험', '포르노', '법', '요인', '경고', '비판', '콘텐츠', '대응', '실제', '연구', '국내', '해당', '필요', '경쟁', '식별', '강화', '배포', '정권', '미국', '배경', '알고리즘', '시행', '회로', '대통령', '진짜', '영상', '개', '위', '일', '집무', '진행', '확장', '버락', '유튜브', '오바마', '마이크로소프트', '트레이스', '페이스북', '버즈피드', '생성자', '페이크', '구글', '도널드', '', '제작', '딥', '트럼프', '발견', '월']


### 문제 생성

In [71]:
question_arr = []
result_arr = []
for question in data['summary_sentences'][2]:
    for word in blank_arr:
        question_tmp = question.replace(word,"*"*len(word))
        if question_tmp != question:
            question_arr.append(question_tmp)
            result_arr.append(word)
            break

for idx in range(len(question_arr)):
    print(question_arr[idx]+"\n정답: "+result_arr[idx])
    print()

해당 영상에는 미국 전 대통령 버락 오바마가 집무실 배경으로 도널드 트럼프 정권을 **하고 있는데, 이는 진짜 영상이 아닌 딥페이크 영상이었다.
정답: 비판

딥트레이스는 2018년 한해 7964개의 딥페이크 **을 발견했다.
정답: 영상

2018년 4월 미국 콘텐츠 제작사인 버즈피드(BuzzFeed)는 이러한 **성을 경고하고자 딥페이크 영상을 유튜브에 올린 바 있다.
정답: 위험

GAN 알고리즘은 생성자를 위해 식별자라는 **을 넣었는데, 딥페이크 대응을 위해서는 식별자를 강화하는 연구도 필요하다.
정답: 요인

GAN 알고리즘에서의 두 ** 경쟁이 실제 사회로까지 확장된 셈이다.
정답: 요인

이에 따라, 국내에서는 딥페이크 포르노 제작 및 배포를 **하는 법이 25일부터 시행된다.
정답: 처벌

페이스북, 마이크로소프트, 구글 등이 앞장서 이를 식별하는 **를 진행하고 있다.
정답: 연구

